In [252]:
# case study
import json

analyze_row_index = 83
file_path = 'report/report.json'
with open(file_path, 'r') as file:
    data = json.load(file)
data[analyze_row_index]['sents']

[['Ashwathy',
  'Kurup',
  ',',
  'better',
  'known',
  'by',
  'her',
  'stage',
  'name',
  'Parvathy',
  ',',
  'is',
  'an',
  'Indian',
  'film',
  'actress',
  'and',
  'classical',
  'dancer',
  ',',
  'who',
  'appeared',
  'in',
  'Malayalam',
  'films',
  '.'],
 ['Parvathy',
  'was',
  'a',
  'popular',
  'actress',
  'in',
  'Malayalam',
  'cinema',
  'during',
  'the',
  'late-1980s',
  'and',
  'early-1990s',
  '.'],
 ['Her',
  'first',
  'film',
  'was',
  'directed',
  'by',
  'Lenin',
  'Rajendran',
  ',',
  'but',
  'was',
  'shelved',
  'and',
  'never',
  'released',
  '.'],
 ['She',
  'was',
  'introduced',
  'to',
  'the',
  'industry',
  'by',
  'actor',
  '-',
  'director',
  'Balachandra',
  'Menon',
  'through',
  'Vivahithare',
  'Ithile',
  'in',
  '1986',
  '.'],
 ['Her',
  'notable',
  'works',
  'include',
  'Amrutham',
  'Gamaya',
  ',',
  'Oru',
  'Minnaminunginte',
  'Nurunguvettam',
  ',',
  'Thoovanathumbikal',
  '(',
  '1987',
  ')',
  ',',
  'Ponmu

In [253]:
def span_to_words(sents, span) -> str:
    index_of_sent = span[0][0]
    start = span[0][1]
    end = span[1][1]
    words = sents[index_of_sent][start:end]
    return ' '.join(words)

span_to_words(data[analyze_row_index]['sents'], data[analyze_row_index]['raw_spans'][0])

'Ashwathy Kurup'

In [254]:
def join_sents_words(sents):
    return [' '.join(sent) for sent in sents]
join_sents_words(data[analyze_row_index]['sents'])

['Ashwathy Kurup , better known by her stage name Parvathy , is an Indian film actress and classical dancer , who appeared in Malayalam films .',
 'Parvathy was a popular actress in Malayalam cinema during the late-1980s and early-1990s .',
 'Her first film was directed by Lenin Rajendran , but was shelved and never released .',
 'She was introduced to the industry by actor - director Balachandra Menon through Vivahithare Ithile in 1986 .',
 'Her notable works include Amrutham Gamaya , Oru Minnaminunginte Nurunguvettam , Thoovanathumbikal ( 1987 ) , Ponmuttayidunna Tharavu ( 1988 ) , Vadakkunokkiyantram , Peruvannapurathe Visheshangal and Kireedam ( 1989 ) .',
 'Parvathy married film actor Jayaram who was her co - star in many films on',
 '7th September 1992 at Town Hall , Ernakulam .',
 'After marriage , Parvathy effectively quit acting in films .',
 'She now lives with her family in Chennai .',
 'She has two children , Kalidas Jayaram and Malavika Jayaram .']

In [255]:
def cluster_to_words(sents, span, cluster):
    words = []
    spans = []
    for i in cluster:
        if i != -1:
            words.append(span_to_words(sents, span[i]))
            spans.append(span[i])
        else:
            words.append('None')
            spans.append('None')
    return words, spans

cluster_to_words(data[analyze_row_index]['sents'], data[analyze_row_index]['raw_spans'], data[analyze_row_index]['clusters_pred'][0])

(['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Parvathy', 'Parvathy'],
 [[[0, 0], [0, 2]],
  [[0, 9], [0, 10]],
  [[1, 0], [1, 1]],
  [[5, 0], [5, 1]],
  [[7, 3], [7, 4]]])

In [256]:
import json
rel_info_path = 'data/docred/rel_info.json'
rel2id_path = 'data/docred/rel2id.json'
with open(rel_info_path, 'r') as file:
    rel_info = json.load(file)
with open(rel2id_path, 'r') as file:
    rel2id = json.load(file)

id2rel_info = {}
for k, v in rel2id.items():
    if v != 0:
        id2rel_info[v] = rel_info[k]
id2rel = {}
for k, v in rel2id.items():
    if v != 0:
        id2rel[v] = k

def visualize_rel(sents, span, rel, clusters, id2rel_info: dict, evidence :list=None):
    if(rel['h'] != -1):
        head = cluster_to_words(sents, span, clusters[rel['h']])
    else:
        head = 'None'
    if(rel['t'] != -1):
        tail = cluster_to_words(sents, span, clusters[rel['t']])
    else:
        tail = 'None'
    rel_info = id2rel_info[rel['r']]
    v_rel = {}
    v_rel['h'] = head
    v_rel['t'] = tail
    v_rel['r'] = rel_info
    if evidence != None:
        v_rel['evidence'] = evidence
    return v_rel

visualize_rel(data[analyze_row_index]['sents'], data[analyze_row_index]['raw_spans'], data[analyze_row_index]['rels_gold'][0], data[analyze_row_index]['clusters_gold'], id2rel_info)

{'h': (['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Parvathy', 'Parvathy'],
  [[[0, 0], [0, 2]],
   [[7, 3], [7, 4]],
   [[5, 0], [5, 1]],
   [[1, 0], [1, 1]],
   [[0, 9], [0, 10]]]),
 't': (['Jayaram'], [[[5, 4], [5, 5]]]),
 'r': 'spouse'}

In [257]:
def find_evidence(rel, id2rel, labels):
    rel_h = rel['h']
    rel_t = rel['t']
    rel_r = id2rel[rel['r']]
    for i in labels:
        if rel_h==i['h'] and rel_t==i['t'] and rel_r==i['r']:
            return i['evidence']
    return None
        

In [258]:
def transform_row(row, id2rel_info, id2rel):
    result_dict = {}
    result_dict['sents'] = join_sents_words(row['sents'])
    num_sents = len(result_dict['sents'])
    result_dict['num_sents'] = num_sents
    words = 0
    for sent in row['sents']:
        words += len(sent)
    result_dict['words'] = words
    result_dict['re_tp'] = row['re_tp']
    result_dict['re_fp'] = row['re_fp']
    result_dict['re_fn'] = row['re_fn']
    result_dict['reverse_cluster_map'] = row['reverse_cluster_map']
    result_dict['cnt_rel_could_not_be_predicted_me'] = row['cnt_rel_could_not_be_predicted_me']
    result_dict['cnt_rel_could_not_be_predicted_coref'] = row['cnt_rel_could_not_be_predicted_coref']
    
    rels_pred = row['rels_pred']
    rels_gold = row['rels_gold']
    clusters_gold = row['clusters_gold']
    predict_right_rel = []
    predict_wrong_rel_by_missing_head_tail = []
    predict_wrong_rel_by_missing_head = []
    predict_wrong_rel_by_missing_tail = []
    predict_wrong_rel_left = []
    cluster_map = row['cluster_map']
    golden_rel_predicted_indexs = []
    for rel in rels_pred:
        mapped_rel = {'h': cluster_map[str(rel['h'])], 't': cluster_map[str(rel['t'])], 'r': rel['r']}
        v_rel = visualize_rel(row['sents'], row['raw_spans'], rel, row['clusters_pred'], id2rel_info)
        if mapped_rel in rels_gold:
            predict_right_rel.append(v_rel)
            golden_rel_predicted_indexs.append(rels_gold.index(mapped_rel))
        elif mapped_rel['h'] == -1 and mapped_rel['t'] == -1:
            predict_wrong_rel_by_missing_head_tail.append(v_rel)
        elif mapped_rel['h'] == -1 :
            predict_wrong_rel_by_missing_head.append(v_rel)
        elif mapped_rel['t'] == -1 :
            predict_wrong_rel_by_missing_tail.append(v_rel)
        else:
            predict_wrong_rel_left.append(v_rel)
    result_dict['predict_right_rel'] = predict_right_rel
    result_dict['predict_wrong_rel_by_missing_head_tail'] = predict_wrong_rel_by_missing_head_tail
    result_dict['predict_wrong_rel_by_missing_head'] = predict_wrong_rel_by_missing_head
    result_dict['predict_wrong_rel_by_missing_tail'] = predict_wrong_rel_by_missing_tail
    result_dict['predict_wrong_rel_left'] = predict_wrong_rel_left
    
    # rel_gold
    golden_rel_predicted = []
    golden_rel_missing_span = []
    golden_rel_missing_head_and_tail = []
    golden_rel_missing_head = []
    golden_rel_missing_tail = []
    golden_rel_not_predicted = []
    labels = row['labels'] if 'labels' in row else []
    reverse_cluster_map = result_dict['reverse_cluster_map']
    for index, rel in enumerate(rels_gold):
        evidence = find_evidence(rel, id2rel, labels)
        v_rel = visualize_rel(row['sents'], row['raw_spans'], rel, row['clusters_gold'], id2rel_info, evidence)
        if index in golden_rel_predicted_indexs:
            golden_rel_predicted.append(v_rel)
        elif -1 in clusters_gold[rel['h']] or -1 in clusters_gold[rel['t']] :
            golden_rel_missing_span.append(v_rel)
        elif reverse_cluster_map[str(rel['h'])] == -1 and reverse_cluster_map[str(rel['t'])] == -1 :
            golden_rel_missing_head_and_tail.append(v_rel)
        elif reverse_cluster_map[str(rel['h'])] == -1:
            golden_rel_missing_head.append(v_rel)
        elif reverse_cluster_map[str(rel['t'])] == -1:
            golden_rel_missing_tail.append(v_rel)
        else:
            golden_rel_not_predicted.append(v_rel)
    result_dict['golden_rel_predicted'] = golden_rel_predicted
    result_dict['golden_rel_missing_span'] = golden_rel_missing_span
    result_dict['golden_rel_missing_head_and_tail'] = golden_rel_missing_head_and_tail
    result_dict['golden_rel_missing_head'] = golden_rel_missing_head
    result_dict['golden_rel_missing_tail'] = golden_rel_missing_tail
    result_dict['golden_rel_not_predicted'] = golden_rel_not_predicted
    golden_rel_predicted_cnt_evidence_0 = 0
    golden_rel_predicted_cnt_evidence_1 = 0
    golden_rel_predicted_cnt_evidence_2 = 0
    golden_rel_predicted_cnt_evidence_3 = 0
    golden_rel_predicted_cnt_evidence_4 = 0
    golden_rel_predicted_cnt_evidence_5 = 0
    golden_rel_predicted_cnt_evidence_6 = 0  # more than 6
    for rel in golden_rel_predicted:
        if 'evidence' in rel:
            if len(rel['evidence']) == 0:
                golden_rel_predicted_cnt_evidence_0 += 1
            elif len(rel['evidence']) == 1:
                golden_rel_predicted_cnt_evidence_1 += 1
            elif len(rel['evidence']) == 2:
                golden_rel_predicted_cnt_evidence_2 += 1
            elif len(rel['evidence']) == 3:
                golden_rel_predicted_cnt_evidence_3 += 1
            elif len(rel['evidence']) == 4:
                golden_rel_predicted_cnt_evidence_4 += 1
            elif len(rel['evidence']) == 5:
                golden_rel_predicted_cnt_evidence_5 += 1
            else:
                golden_rel_predicted_cnt_evidence_6 += 1
    result_dict['golden_rel_predicted_cnt_evidence_0'] = golden_rel_predicted_cnt_evidence_0
    result_dict['golden_rel_predicted_cnt_evidence_1'] = golden_rel_predicted_cnt_evidence_1
    result_dict['golden_rel_predicted_cnt_evidence_2'] = golden_rel_predicted_cnt_evidence_2
    result_dict['golden_rel_predicted_cnt_evidence_3'] = golden_rel_predicted_cnt_evidence_3
    result_dict['golden_rel_predicted_cnt_evidence_4'] = golden_rel_predicted_cnt_evidence_4
    result_dict['golden_rel_predicted_cnt_evidence_5'] = golden_rel_predicted_cnt_evidence_5
    result_dict['golden_rel_predicted_cnt_evidence_6'] = golden_rel_predicted_cnt_evidence_6
    
    golden_rel_missing_cnt_evidence_0 = 0
    golden_rel_missing_cnt_evidence_1 = 0
    golden_rel_missing_cnt_evidence_2 = 0
    golden_rel_missing_cnt_evidence_3 = 0
    golden_rel_missing_cnt_evidence_4 = 0
    golden_rel_missing_cnt_evidence_5 = 0
    golden_rel_missing_cnt_evidence_6 = 0
    
    for rel in golden_rel_missing_span:
        if 'evidence' in rel:
            if len(rel['evidence']) == 0:
                golden_rel_missing_cnt_evidence_0 += 1
            elif len(rel['evidence']) == 1:
                golden_rel_missing_cnt_evidence_1 += 1
            elif len(rel['evidence']) == 2:
                golden_rel_missing_cnt_evidence_2 += 1
            elif len(rel['evidence']) == 3:
                golden_rel_missing_cnt_evidence_3 += 1
            elif len(rel['evidence']) == 4:
                golden_rel_missing_cnt_evidence_4 += 1
            elif len(rel['evidence']) == 5:
                golden_rel_missing_cnt_evidence_5 += 1
            else:
                golden_rel_missing_cnt_evidence_6 += 1
    for rel in golden_rel_missing_head_and_tail:
        if 'evidence' in rel:
            if len(rel['evidence']) == 0:
                golden_rel_missing_cnt_evidence_0 += 1
            elif len(rel['evidence']) == 1:
                golden_rel_missing_cnt_evidence_1 += 1
            elif len(rel['evidence']) == 2:
                golden_rel_missing_cnt_evidence_2 += 1
            elif len(rel['evidence']) == 3:
                golden_rel_missing_cnt_evidence_3 += 1
            elif len(rel['evidence']) == 4:
                golden_rel_missing_cnt_evidence_4 += 1
            elif len(rel['evidence']) == 5:
                golden_rel_missing_cnt_evidence_5 += 1
            else:
                golden_rel_missing_cnt_evidence_6 += 1
    for rel in golden_rel_missing_head:
        if 'evidence' in rel:
            if len(rel['evidence']) == 0:
                golden_rel_missing_cnt_evidence_0 += 1
            elif len(rel['evidence']) == 1:
                golden_rel_missing_cnt_evidence_1 += 1
            elif len(rel['evidence']) == 2:
                golden_rel_missing_cnt_evidence_2 += 1
            elif len(rel['evidence']) == 3:
                golden_rel_missing_cnt_evidence_3 += 1
            elif len(rel['evidence']) == 4:
                golden_rel_missing_cnt_evidence_4 += 1
            elif len(rel['evidence']) == 5:
                golden_rel_missing_cnt_evidence_5 += 1
            else:
                golden_rel_missing_cnt_evidence_6 += 1
    for rel in golden_rel_missing_tail:
        if 'evidence' in rel:
            if len(rel['evidence']) == 0:
                golden_rel_missing_cnt_evidence_0 += 1
            elif len(rel['evidence']) == 1:
                golden_rel_missing_cnt_evidence_1 += 1
            elif len(rel['evidence']) == 2:
                golden_rel_missing_cnt_evidence_2 += 1
            elif len(rel['evidence']) == 3:
                golden_rel_missing_cnt_evidence_3 += 1
            elif len(rel['evidence']) == 4:
                golden_rel_missing_cnt_evidence_4 += 1
            elif len(rel['evidence']) == 5:
                golden_rel_missing_cnt_evidence_5 += 1
            else:
                golden_rel_missing_cnt_evidence_6 += 1
    
    result_dict['golden_rel_missing_cnt_evidence_0'] = golden_rel_missing_cnt_evidence_0
    result_dict['golden_rel_missing_cnt_evidence_1'] = golden_rel_missing_cnt_evidence_1
    result_dict['golden_rel_missing_cnt_evidence_2'] = golden_rel_missing_cnt_evidence_2
    result_dict['golden_rel_missing_cnt_evidence_3'] = golden_rel_missing_cnt_evidence_3
    result_dict['golden_rel_missing_cnt_evidence_4'] = golden_rel_missing_cnt_evidence_4
    result_dict['golden_rel_missing_cnt_evidence_5'] = golden_rel_missing_cnt_evidence_5
    result_dict['golden_rel_missing_cnt_evidence_6'] = golden_rel_missing_cnt_evidence_6
    
    golden_rel_not_predicted_cnt_evidence_0 = 0
    golden_rel_not_predicted_cnt_evidence_1 = 0
    golden_rel_not_predicted_cnt_evidence_2 = 0
    golden_rel_not_predicted_cnt_evidence_3 = 0
    golden_rel_not_predicted_cnt_evidence_4 = 0
    golden_rel_not_predicted_cnt_evidence_5 = 0
    golden_rel_not_predicted_cnt_evidence_6 = 0
    for rel in golden_rel_not_predicted:
        if 'evidence' in rel:
            if len(rel['evidence']) == 0:
                golden_rel_not_predicted_cnt_evidence_0 += 1
            elif len(rel['evidence']) == 1:
                golden_rel_not_predicted_cnt_evidence_1 += 1
            elif len(rel['evidence']) == 2:
                golden_rel_not_predicted_cnt_evidence_2 += 1
            elif len(rel['evidence']) == 3:
                golden_rel_not_predicted_cnt_evidence_3 += 1
            elif len(rel['evidence']) == 4:
                golden_rel_not_predicted_cnt_evidence_4 += 1
            elif len(rel['evidence']) == 5:
                golden_rel_not_predicted_cnt_evidence_5 += 1
            else:
                golden_rel_not_predicted_cnt_evidence_6 += 1
    result_dict['golden_rel_not_predicted_cnt_evidence_0'] = golden_rel_not_predicted_cnt_evidence_0
    result_dict['golden_rel_not_predicted_cnt_evidence_1'] = golden_rel_not_predicted_cnt_evidence_1
    result_dict['golden_rel_not_predicted_cnt_evidence_2'] = golden_rel_not_predicted_cnt_evidence_2
    result_dict['golden_rel_not_predicted_cnt_evidence_3'] = golden_rel_not_predicted_cnt_evidence_3
    result_dict['golden_rel_not_predicted_cnt_evidence_4'] = golden_rel_not_predicted_cnt_evidence_4
    result_dict['golden_rel_not_predicted_cnt_evidence_5'] = golden_rel_not_predicted_cnt_evidence_5
    result_dict['golden_rel_not_predicted_cnt_evidence_6'] = golden_rel_not_predicted_cnt_evidence_6

    return result_dict


In [259]:
row = data[analyze_row_index]

transform_result = transform_row(row, id2rel_info, id2rel)
print(f"cnt_rel_could_not_be_predicted_me: {transform_result['cnt_rel_could_not_be_predicted_me']}")
print(f"cnt_rel_could_not_be_predicted_coref: {transform_result['cnt_rel_could_not_be_predicted_coref']}")

num_sents = transform_result['num_sents']
words = transform_result['words']
print(f'there are {num_sents} sentences, {words} words in doc.')
print(f"sents:")
transform_result['sents']

cnt_rel_could_not_be_predicted_me: 9
cnt_rel_could_not_be_predicted_coref: 3
there are 10 sentences, 160 words in doc.
sents:


['Ashwathy Kurup , better known by her stage name Parvathy , is an Indian film actress and classical dancer , who appeared in Malayalam films .',
 'Parvathy was a popular actress in Malayalam cinema during the late-1980s and early-1990s .',
 'Her first film was directed by Lenin Rajendran , but was shelved and never released .',
 'She was introduced to the industry by actor - director Balachandra Menon through Vivahithare Ithile in 1986 .',
 'Her notable works include Amrutham Gamaya , Oru Minnaminunginte Nurunguvettam , Thoovanathumbikal ( 1987 ) , Ponmuttayidunna Tharavu ( 1988 ) , Vadakkunokkiyantram , Peruvannapurathe Visheshangal and Kireedam ( 1989 ) .',
 'Parvathy married film actor Jayaram who was her co - star in many films on',
 '7th September 1992 at Town Hall , Ernakulam .',
 'After marriage , Parvathy effectively quit acting in films .',
 'She now lives with her family in Chennai .',
 'She has two children , Kalidas Jayaram and Malavika Jayaram .']

In [260]:
# pred
# visualize entity (mentions)
clusters_pred = row['clusters_pred']
for cluster in clusters_pred:
    print(cluster_to_words(row['sents'], row['raw_spans'], cluster))

(['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Parvathy', 'Parvathy'], [[[0, 0], [0, 2]], [[0, 9], [0, 10]], [[1, 0], [1, 1]], [[5, 0], [5, 1]], [[7, 3], [7, 4]]])
(['Malayalam', 'Malayalam'], [[[0, 23], [0, 24]], [[1, 6], [1, 7]]])
(['Oru Minnaminunginte Nurunguvettam ,'], [[[4, 7], [4, 11]]])
(['Peruvannapurathe Visheshangal and Kireedam'], [[[4, 24], [4, 28]]])
(['1989'], [[[4, 29], [4, 30]]])
(['Kalidas Jayaram'], [[[9, 5], [9, 7]]])
(['Malavika Jayaram'], [[[9, 8], [9, 10]]])
(['two'], [[[9, 2], [9, 3]]])
(['Ernakulam'], [[[6, 7], [6, 8]]])
(['Vivahithare Ithile'], [[[3, 13], [3, 15]]])
(['1988'], [[[4, 19], [4, 20]]])
(['Town Hall'], [[[6, 4], [6, 6]]])
(['1987'], [[[4, 13], [4, 14]]])
(['Jayaram'], [[[5, 4], [5, 5]]])
(['Amrutham Gamaya'], [[[4, 4], [4, 6]]])
(['Ponmuttayidunna Tharavu'], [[[4, 16], [4, 18]]])
(['Chennai'], [[[8, 7], [8, 8]]])
(['Vadakkunokkiyantram'], [[[4, 22], [4, 23]]])
(['7th September 1992'], [[[6, 0], [6, 3]]])
(['late-1980s'], [[[1, 10], [1, 11]]])
(['Leni

In [261]:
# gold
# visualize entity (mentions)
clusters_gold = row['clusters_gold']
for cluster in clusters_gold:
    print(cluster_to_words(row['sents'], row['raw_spans'], cluster))

(['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Parvathy', 'Parvathy'], [[[0, 0], [0, 2]], [[7, 3], [7, 4]], [[5, 0], [5, 1]], [[1, 0], [1, 1]], [[0, 9], [0, 10]]])
(['Indian'], [[[0, 13], [0, 14]]])
(['Malayalam'], [[[0, 23], [0, 24]]])
(['Malayalam'], [[[1, 6], [1, 7]]])
(['late-1980s'], [[[1, 10], [1, 11]]])
(['early-1990s'], [[[1, 12], [1, 13]]])
(['Lenin Rajendran'], [[[2, 6], [2, 8]]])
(['Balachandra Menon'], [[[3, 10], [3, 12]]])
(['Vivahithare Ithile'], [[[3, 13], [3, 15]]])
(['1986'], [[[3, 16], [3, 17]]])
(['Amrutham Gamaya'], [[[4, 4], [4, 6]]])
(['None'], ['None'])
(['Thoovanathumbikal'], [[[4, 11], [4, 12]]])
(['1987'], [[[4, 13], [4, 14]]])
(['Ponmuttayidunna Tharavu'], [[[4, 16], [4, 18]]])
(['1988'], [[[4, 19], [4, 20]]])
(['Vadakkunokkiyantram'], [[[4, 22], [4, 23]]])
(['None'], ['None'])
(['None'], ['None'])
(['1989'], [[[4, 29], [4, 30]]])
(['Jayaram'], [[[5, 4], [5, 5]]])
(['7th September 1992'], [[[6, 0], [6, 3]]])
(['Town Hall'], [[[6, 4], [6, 6]]])
(['Ernakulam'], [

In [262]:
print(f"predict_right_rel: {len(transform_result['predict_right_rel'])}")
transform_result['predict_right_rel']

predict_right_rel: 9


[{'h': (['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Parvathy', 'Parvathy'],
   [[[0, 0], [0, 2]],
    [[0, 9], [0, 10]],
    [[1, 0], [1, 1]],
    [[5, 0], [5, 1]],
    [[7, 3], [7, 4]]]),
  't': (['Kalidas Jayaram'], [[[9, 5], [9, 7]]]),
  'r': 'child'},
 {'h': (['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Parvathy', 'Parvathy'],
   [[[0, 0], [0, 2]],
    [[0, 9], [0, 10]],
    [[1, 0], [1, 1]],
    [[5, 0], [5, 1]],
    [[7, 3], [7, 4]]]),
  't': (['Malavika Jayaram'], [[[9, 8], [9, 10]]]),
  'r': 'child'},
 {'h': (['Jayaram'], [[[5, 4], [5, 5]]]),
  't': (['Kalidas Jayaram'], [[[9, 5], [9, 7]]]),
  'r': 'child'},
 {'h': (['Amrutham Gamaya'], [[[4, 4], [4, 6]]]),
  't': (['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Parvathy', 'Parvathy'],
   [[[0, 0], [0, 2]],
    [[0, 9], [0, 10]],
    [[1, 0], [1, 1]],
    [[5, 0], [5, 1]],
    [[7, 3], [7, 4]]]),
  'r': 'cast member'},
 {'h': (['Ponmuttayidunna Tharavu'], [[[4, 16], [4, 18]]]),
  't': (['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Par

In [263]:
print(f"predict_wrong_rel_by_missing_head_tail: {len(transform_result['predict_wrong_rel_by_missing_head_tail'])}")
transform_result['predict_wrong_rel_by_missing_head_tail']

predict_wrong_rel_by_missing_head_tail: 0


[]

In [264]:

print(f"predict_wrong_rel_by_missing_head: {len(transform_result['predict_wrong_rel_by_missing_head'])}")
transform_result['predict_wrong_rel_by_missing_head']

predict_wrong_rel_by_missing_head: 2


[{'h': (['Oru Minnaminunginte Nurunguvettam ,'], [[[4, 7], [4, 11]]]),
  't': (['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Parvathy', 'Parvathy'],
   [[[0, 0], [0, 2]],
    [[0, 9], [0, 10]],
    [[1, 0], [1, 1]],
    [[5, 0], [5, 1]],
    [[7, 3], [7, 4]]]),
  'r': 'cast member'},
 {'h': (['Peruvannapurathe Visheshangal and Kireedam'], [[[4, 24], [4, 28]]]),
  't': (['1989'], [[[4, 29], [4, 30]]]),
  'r': 'publication date'}]

In [265]:
print(f"predict_wrong_rel_by_missing_tail: {len(transform_result['predict_wrong_rel_by_missing_tail'])}")
transform_result['predict_wrong_rel_by_missing_tail']

predict_wrong_rel_by_missing_tail: 1


[{'h': (['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Parvathy', 'Parvathy'],
   [[[0, 0], [0, 2]],
    [[0, 9], [0, 10]],
    [[1, 0], [1, 1]],
    [[5, 0], [5, 1]],
    [[7, 3], [7, 4]]]),
  't': (['Malayalam', 'Malayalam'], [[[0, 23], [0, 24]], [[1, 6], [1, 7]]]),
  'r': 'languages spoken, written or signed'}]

In [266]:
print(f"predict_wrong_rel_left: {len(transform_result['predict_wrong_rel_left'])}")
transform_result['predict_wrong_rel_left']

predict_wrong_rel_left: 3


[{'h': (['Kalidas Jayaram'], [[[9, 5], [9, 7]]]),
  't': (['Malavika Jayaram'], [[[9, 8], [9, 10]]]),
  'r': 'sibling'},
 {'h': (['Malavika Jayaram'], [[[9, 8], [9, 10]]]),
  't': (['Kalidas Jayaram'], [[[9, 5], [9, 7]]]),
  'r': 'sibling'},
 {'h': (['Jayaram'], [[[5, 4], [5, 5]]]),
  't': (['Malavika Jayaram'], [[[9, 8], [9, 10]]]),
  'r': 'child'}]

In [267]:
print(f"golden_rel_predicted: {len(transform_result['golden_rel_predicted'])}")
transform_result['golden_rel_predicted']

golden_rel_predicted: 9


[{'h': (['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Parvathy', 'Parvathy'],
   [[[0, 0], [0, 2]],
    [[7, 3], [7, 4]],
    [[5, 0], [5, 1]],
    [[1, 0], [1, 1]],
    [[0, 9], [0, 10]]]),
  't': (['Kalidas Jayaram'], [[[9, 5], [9, 7]]]),
  'r': 'child',
  'evidence': [0, 9]},
 {'h': (['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Parvathy', 'Parvathy'],
   [[[0, 0], [0, 2]],
    [[7, 3], [7, 4]],
    [[5, 0], [5, 1]],
    [[1, 0], [1, 1]],
    [[0, 9], [0, 10]]]),
  't': (['Malavika Jayaram'], [[[9, 8], [9, 10]]]),
  'r': 'child',
  'evidence': [0, 9]},
 {'h': (['Jayaram'], [[[5, 4], [5, 5]]]),
  't': (['Kalidas Jayaram'], [[[9, 5], [9, 7]]]),
  'r': 'child',
  'evidence': [9, 5]},
 {'h': (['Amrutham Gamaya'], [[[4, 4], [4, 6]]]),
  't': (['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Parvathy', 'Parvathy'],
   [[[0, 0], [0, 2]],
    [[7, 3], [7, 4]],
    [[5, 0], [5, 1]],
    [[1, 0], [1, 1]],
    [[0, 9], [0, 10]]]),
  'r': 'cast member',
  'evidence': [0, 4]},
 {'h': (['Thoovanathumbikal

In [268]:
print(f"golden_rel_missing_span: {len(transform_result['golden_rel_missing_span'])}")
transform_result['golden_rel_missing_span']

golden_rel_missing_span: 9


[{'h': (['None'], ['None']),
  't': (['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Parvathy', 'Parvathy'],
   [[[0, 0], [0, 2]],
    [[7, 3], [7, 4]],
    [[5, 0], [5, 1]],
    [[1, 0], [1, 1]],
    [[0, 9], [0, 10]]]),
  'r': 'cast member',
  'evidence': [0, 4]},
 {'h': (['None'], ['None']),
  't': (['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Parvathy', 'Parvathy'],
   [[[0, 0], [0, 2]],
    [[7, 3], [7, 4]],
    [[5, 0], [5, 1]],
    [[1, 0], [1, 1]],
    [[0, 9], [0, 10]]]),
  'r': 'cast member',
  'evidence': [0, 4]},
 {'h': (['None'], ['None']),
  't': (['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Parvathy', 'Parvathy'],
   [[[0, 0], [0, 2]],
    [[7, 3], [7, 4]],
    [[5, 0], [5, 1]],
    [[1, 0], [1, 1]],
    [[0, 9], [0, 10]]]),
  'r': 'cast member',
  'evidence': [0, 4]},
 {'h': (['None'], ['None']),
  't': (['1989'], [[[4, 29], [4, 30]]]),
  'r': 'publication date',
  'evidence': [4]},
 {'h': (['None'], ['None']),
  't': (['Malayalam'], [[[0, 23], [0, 24]]]),
  'r': 'original la

In [269]:
print(f"golden_rel_missing_head_and_tail: {len(transform_result['golden_rel_missing_head_and_tail'])}")
transform_result['golden_rel_missing_head_and_tail']

golden_rel_missing_head_and_tail: 0


[]

In [270]:
print(f"golden_rel_missing_head: {len(transform_result['golden_rel_missing_head'])}")
transform_result['golden_rel_missing_head']

golden_rel_missing_head: 0


[]

In [271]:
print(f"golden_rel_missing_tail: {len(transform_result['golden_rel_missing_tail'])}")
transform_result['golden_rel_missing_tail']

golden_rel_missing_tail: 3


[{'h': (['Thoovanathumbikal'], [[[4, 11], [4, 12]]]),
  't': (['Malayalam'], [[[0, 23], [0, 24]]]),
  'r': 'original language of work',
  'evidence': [0, 1, 4]},
 {'h': (['Ponmuttayidunna Tharavu'], [[[4, 16], [4, 18]]]),
  't': (['Malayalam'], [[[0, 23], [0, 24]]]),
  'r': 'original language of work',
  'evidence': [0, 1, 4]},
 {'h': (['Vadakkunokkiyantram'], [[[4, 22], [4, 23]]]),
  't': (['Malayalam'], [[[0, 23], [0, 24]]]),
  'r': 'original language of work',
  'evidence': [0, 1, 4]}]

In [272]:
print(f"golden_rel_not_predicted: {len(transform_result['golden_rel_not_predicted'])}")
transform_result['golden_rel_not_predicted']

golden_rel_not_predicted: 8


[{'h': (['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Parvathy', 'Parvathy'],
   [[[0, 0], [0, 2]],
    [[7, 3], [7, 4]],
    [[5, 0], [5, 1]],
    [[1, 0], [1, 1]],
    [[0, 9], [0, 10]]]),
  't': (['Jayaram'], [[[5, 4], [5, 5]]]),
  'r': 'spouse',
  'evidence': [5]},
 {'h': (['Jayaram'], [[[5, 4], [5, 5]]]),
  't': (['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Parvathy', 'Parvathy'],
   [[[0, 0], [0, 2]],
    [[7, 3], [7, 4]],
    [[5, 0], [5, 1]],
    [[1, 0], [1, 1]],
    [[0, 9], [0, 10]]]),
  'r': 'spouse',
  'evidence': [5]},
 {'h': (['Kalidas Jayaram'], [[[9, 5], [9, 7]]]),
  't': (['Ashwathy Kurup', 'Parvathy', 'Parvathy', 'Parvathy', 'Parvathy'],
   [[[0, 0], [0, 2]],
    [[7, 3], [7, 4]],
    [[5, 0], [5, 1]],
    [[1, 0], [1, 1]],
    [[0, 9], [0, 10]]]),
  'r': 'mother',
  'evidence': [0, 9]},
 {'h': (['Kalidas Jayaram'], [[[9, 5], [9, 7]]]),
  't': (['Jayaram'], [[[5, 4], [5, 5]]]),
  'r': 'father',
  'evidence': [9, 5]},
 {'h': (['Vivahithare Ithile'], [[[3, 13], [3, 15]]])

In [273]:
# transformer data
dict_list_to_be_save = []
for row in data:
    dict_list_to_be_save.append(transform_row(row, id2rel_info, id2rel))

In [274]:
# # save as csv
# save_path = 'report/csv_report.csv'

# import csv
# fieldnames = set().union(*(d.keys() for d in dict_list_to_be_save))
# # 使用字典的键作为CSV的列头
# with open(save_path, 'w', newline='') as csvfile:
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()
#     writer.writerows(dict_list_to_be_save)

# # save as json
# json_file_path = "report/json_report.json"
# with open(json_file_path, "w") as f:
#     json.dump(dict_list_to_be_save, f)

In [275]:
def caculate_mentions_rels(transformed_data):
    gold_rel_predicted_right_cnt_mention_2 = 0
    gold_rel_predicted_right_cnt_mention_3 = 0
    gold_rel_predicted_right_cnt_mention_4 = 0
    gold_rel_predicted_right_cnt_mention_5 = 0
    
    gold_rel_predicted_wrong_left_cnt_mention_2 = 0
    gold_rel_predicted_wrong_left_cnt_mention_3 = 0
    gold_rel_predicted_wrong_left_cnt_mention_4 = 0
    gold_rel_predicted_wrong_left_cnt_mention_5 = 0
    
    for row in transformed_data:
        for rel in row['golden_rel_predicted']:
            if len(rel['h'][0]) + len(rel['t'][0]) == 2:
                gold_rel_predicted_right_cnt_mention_2 += 1
            elif len(rel['h'][0]) + len(rel['t'][0]) == 3:
                gold_rel_predicted_right_cnt_mention_3 += 1
            elif len(rel['h'][0]) + len(rel['t'][0]) == 4:
                gold_rel_predicted_right_cnt_mention_4 += 1
            elif len(rel['h'][0]) + len(rel['t'][0]) >= 5:
                gold_rel_predicted_right_cnt_mention_5 += 1
        for rel in row['golden_rel_not_predicted']:
            if len(rel['h'][0]) + len(rel['t'][0]) == 2:
                gold_rel_predicted_wrong_left_cnt_mention_2 += 1
            elif len(rel['h'][0]) + len(rel['t'][0]) == 3:
                gold_rel_predicted_wrong_left_cnt_mention_3 += 1
            elif len(rel['h'][0]) + len(rel['t'][0]) == 4:
                gold_rel_predicted_wrong_left_cnt_mention_4 += 1
            elif len(rel['h'][0]) + len(rel['t'][0]) >= 5:
                gold_rel_predicted_wrong_left_cnt_mention_5 += 1
    
    result = {}
    result['gold_rel_predicted_right_cnt_mention_2'] = gold_rel_predicted_right_cnt_mention_2
    result['gold_rel_predicted_right_cnt_mention_3'] = gold_rel_predicted_right_cnt_mention_3
    result['gold_rel_predicted_right_cnt_mention_4'] = gold_rel_predicted_right_cnt_mention_4
    result['gold_rel_predicted_right_cnt_mention_5'] = gold_rel_predicted_right_cnt_mention_5
    
    result['gold_rel_predicted_wrong_left_cnt_mention_2'] = gold_rel_predicted_wrong_left_cnt_mention_2
    result['gold_rel_predicted_wrong_left_cnt_mention_3'] = gold_rel_predicted_wrong_left_cnt_mention_3
    result['gold_rel_predicted_wrong_left_cnt_mention_4'] = gold_rel_predicted_wrong_left_cnt_mention_4
    result['gold_rel_predicted_wrong_left_cnt_mention_5'] = gold_rel_predicted_wrong_left_cnt_mention_5
    return result

result = caculate_mentions_rels(dict_list_to_be_save)
result

{'gold_rel_predicted_right_cnt_mention_2': 2119,
 'gold_rel_predicted_right_cnt_mention_3': 1008,
 'gold_rel_predicted_right_cnt_mention_4': 681,
 'gold_rel_predicted_right_cnt_mention_5': 888,
 'gold_rel_predicted_wrong_left_cnt_mention_2': 1595,
 'gold_rel_predicted_wrong_left_cnt_mention_3': 766,
 'gold_rel_predicted_wrong_left_cnt_mention_4': 435,
 'gold_rel_predicted_wrong_left_cnt_mention_5': 577}

In [276]:
def caculate_rels(transformed_data):
    golden_rel_predicted_each_rel_cnt = {}
    golden_rel_not_predicted_each_rel_cnt = {}
    for row in transformed_data:
        for rel in row['golden_rel_predicted']:
            if rel['r'] not in golden_rel_predicted_each_rel_cnt:
                golden_rel_predicted_each_rel_cnt[rel['r']] = 1
            else:
                golden_rel_predicted_each_rel_cnt[rel['r']] += 1
        for rel in row['golden_rel_not_predicted']:
            if rel['r'] not in golden_rel_not_predicted_each_rel_cnt:
                golden_rel_not_predicted_each_rel_cnt[rel['r']] = 1
            else:
                golden_rel_not_predicted_each_rel_cnt[rel['r']] += 1
    return golden_rel_predicted_each_rel_cnt, golden_rel_not_predicted_each_rel_cnt

golden_rel_predicted_each_rel_cnt, golden_rel_not_predicted_each_rel_cnt = caculate_rels(dict_list_to_be_save)

# import csv
# each_rel_cnt_csv_report_path = "report/each_rel_cnt.csv"
# headers = sorted(set(golden_rel_predicted_each_rel_cnt.keys()) | set(golden_rel_not_predicted_each_rel_cnt.keys()))
# with open(each_rel_cnt_csv_report_path, 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile)

#     writer.writerow(['', 'golden_rel_predicted', 'golden_rel_not_predicted'])
#     for key in headers:
#         writer.writerow([key, golden_rel_predicted_each_rel_cnt.get(key, '0'), golden_rel_not_predicted_each_rel_cnt.get(key, '0')])
